In [3]:
import os, csv, time
import json
import pandas as pd

import requests
import code.spcxn as sp

In [4]:
# Load token
HEADER,TOKEN = sp.connectSpotify()



            User authentication requires interaction with your
            web browser. Once you enter your credentials and
            give authorization, you will be redirected to
            a url.  Paste that url you were directed to to
            complete the authorization.

        
Opening https://accounts.spotify.com/authorize?client_id=d5eba668c908462aa3183670fe307302&redirect_uri=https%3A%2F%2Flocalhost%3A8888%2F&scope=playlist-read-private&response_type=code in your browser


Enter the URL you were redirected to: https://localhost:8888/?code=AQDQ2kBwujVpmKhG9s-x_hupaqrU3SHQ8HzZzSW-XzhaZN_Ry25DcXNpGAq96koRF4pArAn7AJ-oXg9ZHATIkY1_ySeq4t19DxUg_4rBH8TQldXVAO6ACWAoeGwykQHT5UH7lJa0zt36zElH73EcQ43PBfHKZ2mlmAZYcgrec9a07IjNoakH_unk-r7i7zPDDUZhOYwpJ6c8iEBWfSVmfSKKbRE


Token loaded successfully.


In [5]:
# Load playlist data
PLAYLIST_URL_BASE = 'https://api.spotify.com/v1/playlists/{}'
PLAYLIST_URI = '2PTEg5tENIOnT8b09Bb8IJ'

TRACK_KEYS = {
    'popularity': 'trackPopularity',
    'duration_ms': 'trackDurationMS',
    'explicit': 'trackExplicit',
    'name': 'trackName',
    'id': 'trackID'
}
ALBUM_KEYS = {
    'album_type': 'albumType',
    'release_date_precision': 'albumReleaseDatePrecision',
    'total_tracks': 'albumTotalTracks',
    'name': 'albumName',
    'id': 'albumID',
    'release_date': 'albumReleaseDate'
}

In [6]:
def getPlaylistData(plistData, artistData, counterStart):
    tData, counter = [], counterStart
    for trackCt,i in enumerate(plistData['items']):
        trackData = {
            'added_at': i['added_at'],
            'added_by': i['added_by']['id']
        }

        for tKey,tVal in i['track'].items():
            trackKey = TRACK_KEYS.get(tKey, None)
            if trackKey: trackData[trackKey] = tVal
        trackID = trackData['trackID']
        
        for albKey,albVal in i['track']['album'].items():
            albumKey = ALBUM_KEYS.get(albKey, None)
            if albumKey: trackData[albumKey] = albVal
        
        tData.append(trackData)
        
        
        for artCt,artVal in enumerate(i['track']['artists']):
            artistData['trackID'].append(trackID)
            artistData['artistNo'].append(artCt)
            artistData['artistName'].append(artVal['name'])
            artistData['artistID'].append(artVal['id'])

            
        counter = counter + 1
        
    return (counter,tData,artistData)

In [7]:
firstPass = True
#nextURL = plistDataRaw['tracks']['next']

spotifyRequest = requests.get(PLAYLIST_URL_BASE.format(PLAYLIST_URI), headers=HEADER)

if str(spotifyRequest) == '<Response [200]>':
    rawSpotifyPlistData = spotifyRequest.json()['tracks']
else:
    rawSpotifyPlistData = None

print(spotifyRequest)
artistData = {
    'trackID': [],
    'artistNo': [],
    'artistName': [],
    'artistID': []
}

data,ct = [],1
while rawSpotifyPlistData:
    c,tdat,adat = getPlaylistData(rawSpotifyPlistData, artistData, ct)
    ct = c
    
    # Add the track and artist data
    data = data + tdat
    artistData = adat.copy()
    
    
    if rawSpotifyPlistData['next']:
        nextURI = rawSpotifyPlistData['next']
        nextData = requests.get(nextURI, headers=HEADER)
        rawSpotifyPlistData = nextData.json()
    else:
        rawSpotifyPlistData = None
        
print('Queried {} unique tracks from playlist.'.format(ct))

<Response [200]>
Queried 814 unique tracks from playlist.


In [8]:
keys = list(data[0].keys())
tabularData = [['num'] + keys]

for i,item in enumerate(data):
    lineData = [item[j] for j in keys]
    lineData = [i+1] + lineData
    
    tabularData.append(lineData)
df = pd.DataFrame(tabularData)
df.columns = df.iloc[0]
df = df.drop(0, axis=0)


# write it to csv
fn = 'data/ThingsPlaylist.csv'
df.to_csv(fn, sep='|', quoting=csv.QUOTE_NONNUMERIC, quotechar='`', index=False)

In [9]:
artistDF = pd.DataFrame(artistData)

# write it to csv
fn = 'data/ThingsPlaylist_Artists.csv'
artistDF.to_csv(fn, sep='|', quoting=csv.QUOTE_NONNUMERIC, quotechar='`', index=False)

In [10]:
# Get the genre data
ARTIST_URL = 'https://api.spotify.com/v1/artists/{}'

genres = {
    'artistID': [],
    'genre': [],
    'genreNo': []
}
for counter,aid in enumerate(list(artistDF['artistID'].unique())):
    response = requests.get(ARTIST_URL.format(aid), headers=HEADER)
    
    if str(response) == '<Response [200]>':
        for gno,g in enumerate(response.json()['genres']):
            genres['artistID'].append(aid)
            genres['genre'].append(g)
            genres['genreNo'].append(gno)
    else:
        print('Error loading genres for artist: {}'.format(artistDF['artistName']))
        
    if (counter + 1) % 25 == 0:
        print('Completed querying {} records.'.format(counter + 1))
        
    time.sleep(0.05)

genreDF = pd.DataFrame(genres)
genreDF.head(n=15)

Completed querying 25 records.
Completed querying 50 records.
Completed querying 75 records.
Completed querying 100 records.
Completed querying 125 records.
Completed querying 150 records.
Completed querying 175 records.
Completed querying 200 records.
Completed querying 225 records.
Completed querying 250 records.
Completed querying 275 records.
Completed querying 300 records.
Completed querying 325 records.
Completed querying 350 records.
Completed querying 375 records.
Completed querying 400 records.
Completed querying 425 records.
Completed querying 450 records.
Completed querying 475 records.
Completed querying 500 records.
Completed querying 525 records.
Completed querying 550 records.
Completed querying 575 records.
Completed querying 600 records.
Completed querying 625 records.
Completed querying 650 records.


,artistID,genre,genreNo
0,1Xyo4u8uXC1ZmMpatF05PJ,canadian contemporary r&b,0
1,1Xyo4u8uXC1ZmMpatF05PJ,canadian pop,1
2,1Xyo4u8uXC1ZmMpatF05PJ,pop,2
3,4tZwfgrHOc3mvqYlEYSvVi,electro,0
4,4tZwfgrHOc3mvqYlEYSvVi,filter house,1
5,69GGBxA162lTqCwzJG5jLp,edm,0
6,69GGBxA162lTqCwzJG5jLp,electropop,1
7,69GGBxA162lTqCwzJG5jLp,pop,2
8,69GGBxA162lTqCwzJG5jLp,tropical house,3
9,26VFTg2z8YR0cCuwLzESi2,dance pop,0


In [11]:
fn = 'data/ThingsPlaylist_Genres.csv'
genreDF.to_csv(fn, sep='|', quoting=csv.QUOTE_NONNUMERIC, quotechar='`', index=False)